In [72]:
from __future__ import division,print_function
import pickle
import pandas as pd
import numpy as np


from collections import Counter,defaultdict
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row,column,layout ,widgetbox
from bokeh.models import ColumnDataSource,HoverTool,BoxSelectTool

from bokeh.models import LinearInterpolator
from bokeh.models.mappers import LinearColorMapper
from bokeh.models.widgets import Dropdown,Select
from bokeh.layouts import widgetbox, row, column, layout
from bokeh.io import curdoc

In [54]:
from bokeh.io import output_notebook, push_notebook
output_notebook()

Loading BokehJS ...

In [76]:
df=pd.read_csv('./Predictions_Ensemble.csv')
test_df=pd.read_csv('./TestSet.csv')
print(df.head())
print(test_df.head())
full_df=pd.concat([df.iloc[:,1:],test_df.loc[:,'sentences']],axis=1)
full_df.columns=['dt0','dt1','dt2','dt3','dt4','y_test','sentences']

combined_pred=np.mean(full_df.iloc[:,:5],axis=1)
cpred=[round(p) for p in combined_pred.values]
full_df['cpred']=cpred
full_df['zeros']=[0]*len(cpred)
print(full_df.head())

   Unnamed: 0    0    1    2    3    4  y_test
0           0  1.0  1.0  1.0  1.0  1.0       1
1           1  1.0  1.0  1.0  1.0  1.0       1
2           2  0.0  0.0  0.0  0.0  0.0       0
3           3  1.0  1.0  1.0  1.0  1.0       1
4           4  0.0  0.0  0.0  0.0  0.0       0
   Unnamed: 0       sentences  y_test
0           0    tend analyze       1
1           1  Zamarron Susie       1
2           2   students ITED       0
3           3  Bourke Cockran       1
4           4   budget covers       0
   dt0  dt1  dt2  dt3  dt4  y_test       sentences  cpred  zeros
0  1.0  1.0  1.0  1.0  1.0       1    tend analyze    1.0      0
1  1.0  1.0  1.0  1.0  1.0       1  Zamarron Susie    1.0      0
2  0.0  0.0  0.0  0.0  0.0       0   students ITED    0.0      0
3  1.0  1.0  1.0  1.0  1.0       1  Bourke Cockran    1.0      0
4  0.0  0.0  0.0  0.0  0.0       0   budget covers    0.0      0


In [40]:
print("Combined accuracy: ",np.mean(full_df.loc[:,'y_test'].as_matrix()==full_df.loc[:,'cpred'].as_matrix()))

Combined accuracy:  0.923379860419


In [56]:
t=np.linspace(-10,10,len(full_df))
rs=[]
xs=[]
ys=[]
for times in range(len(t)):
    r=np.random.random()+np.random.random()
    if(r>1):
        rs.append(2-r)
        xs.append((2-r)*np.cos(t[times]))
        ys.append((2-r)*np.sin(t[times]))
    else:
        rs.append(r)
        xs.append(r*np.cos(t[times]))
        ys.append(r*np.sin(t[times]))
len(xs)    

20060

In [77]:
source=ColumnDataSource(data=full_df)
source.add(xs,'xs')
source.add(ys,'ys')
source.add(cpred,'color')
print(source.column_names)

['dt0', 'dt1', 'dt2', 'dt3', 'dt4', 'y_test', 'sentences', 'cpred', 'zeros', 'index', 'xs', 'ys', 'color']


In [79]:
color_mapper=LinearColorMapper(low=0,\
                               high=1,\
                              palette=['red','green'])
select_=None
def select_change(attr,old,new):
      global source,select_,full_df  
      source.data['color']=full_df.loc[:,select_.value]
      print('color changed') 

select_=Select(title="Select Pred",value="cpred",options=["cpred",'dt0','dt1','dt2','dt3','dt4','y_test','zeros'])
select_.on_change('value',select_change)

hover=HoverTool(tooltips=[('Phrase:',"@sentences"),\
                          ('y_test:','@y_test'),\
                          ('cpred','@cpred')],show_arrow=False)

plot=figure(plot_height=500,plot_width=600,x_axis_label='data points',y_axis_label='quality',title='+ve phrases')
plot.add_tools(hover)
plot.circle('xs','ys',source=source,\
           fill_color={'field':'color','transform':color_mapper},\
           size=5)
lay=(row(plot,widgetbox(select_)))
show(lay)
curdoc().add_root(lay)